# YOLOv5 Face Mask Detection 🧪
This notebook walks through loading a trained YOLOv5 model and testing it on images.

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET

from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# --- Step 1: Download Dataset ---
def download_dataset(kaggle_json_path, download_path="data"):
    os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(kaggle_json_path)
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files("andrewmvd/face-mask-detection", path=download_path, unzip=True)
    print("✅ Dataset downloaded and extracted to", download_path)

In [20]:
def create_model(input_shape=(224, 224, 3), num_classes=3):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [21]:
def plot_training(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.legend()
    plt.title('Loss')
    plt.savefig("visualizations/training_plot.png")
    plt.show()

def evaluate_model(model, test_generator):
    Y_pred = model.predict(test_generator)
    y_pred = np.argmax(Y_pred, axis=1)
    print('Classification Report')
    print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices))

    cm = confusion_matrix(test_generator.classes, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices, yticklabels=test_generator.class_indices)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.savefig("visualizations/confusion_matrix.png")
    plt.show()

In [22]:
# Download dataset
kaggle_json_path = "../kaggle.json"  # Adjust path if needed
if not os.path.exists("data/annotations"):  # simple check if already extracted
    download_dataset(kaggle_json_path)

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/face-mask-detection
Dataset downloaded and extracted to data


In [23]:
# Preprocessing
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    'data',
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    'data',
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Found 683 images belonging to 2 classes.
Found 170 images belonging to 2 classes.


In [24]:
# Model
model = create_model(input_shape=img_size + (3,), num_classes=3)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [25]:
# Training
history = model.fit(train_gen, validation_data=val_gen, epochs=10)
model.save("models/mask_detector.h5")

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Ryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\Ryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1046, in launch_instance

  File "c:\Users\Ryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "c:\Users\Ryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Ryan\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\Ryan\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", l

In [ ]:
# Evaluation
plot_training(history)
evaluate_model(model, val_gen)

In [ ]:
# --- Step 6: Evaluate model performance on validation data ---
def evaluate_model(model, val_gen):
    val_gen.reset()
    preds = model.predict(val_gen, verbose=1)
    y_pred = np.argmax(preds, axis=1)
    y_true = val_gen.classes
    class_labels = list(val_gen.class_indices.keys())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_labels))

    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

In [ ]:
# --- Main pipeline ---
def main():
    # Upload file kaggle.json
    upload_kaggle_json()

    # Download dataset dan ekstrak
    download_dataset()

    # Prepare data generators
    train_gen, val_gen = create_data_generators()

    # Build model
    model = build_model()

    # Train model
    print("Mulai training...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=10
    )

    # Plot training history
    plot_history(history)

    # Evaluate model
    evaluate_model(model, val_gen)

if __name__ == '__main__':
    main()